# **1 Linear Algebra**

Using the linear algebra book of your choice. Prove the following statements

  1. Show that if a matrix $A \in \mathbb{R}^{n \times n}$ satisfies $A^2 = 4I_n$, then all eigenvalues of $A$ are 2 and -2.

  2. Let $A$, $B$ $\in \mathbb{R}^{n \times n}$. Proof that the matrix $AB-BA$ can never be positive semidefinite unless it is the zero matrix.

  3. If a matrix $P \in \mathbb{R}^{n \times n}$ satisfies $P^2 = P$, then all its eigenvalues must be 1 or 0.

  4. The columns of the matrix $A \in \mathbb{R}^{n \times d}$ are linearly independent if and only if $c(x) = Ax$ is one-to-one function.

# **2 Write a code**

Using the deep learning library of your choice, solve the following:

  1. The second difference matrix $A \in \mathbb{R}^{n \times n}$ arises in approximating the second derivative on equally spaced data. The formula is:

  $$A_{i,j} =  \Bigg\{ \begin{array}{rcl} -2 & \textrm{if} & j=i  \\ 
  1 & \textrm{if} &  |j-1| = 1 \\ 
  0 & \textrm{if} & |j-1| > 1 \end{array}$$

  Write a code that allows you to create this matrix for a given $n$ and find in the literature the characteristics of this matrix. Then, solve a system of equations $A\textbf{x} = \textbf{b}$ by :

    (a) Calculating the inverse directly.

    (b) Employing a numeric strategy for this particular form of matrices.

    (c) Create a graphs of computational time and memory cost.